In [1]:
# Dependencies
import pandas as pd
import time
import requests as req
from bs4 import BeautifulSoup as bs
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager 

In [2]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\ocean\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


# Scraping

## NASA Mars News

- Scraped the [NASA Mars News Site](https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest) and collected the latest News Title and Paragraph Text.  
- Assign the text to variables that we can reference later.

In [3]:
# set up url for NASA news site and run
news_url = "https://mars.nasa.gov/news/"
browser.visit(news_url)
html = browser.html

# parse with BeautifulSoup
news_soup = bs(html, "html.parser")

# find the latest news article title and its paragraph text
article_title = news_soup.find("div", class_="content_title").text
time.sleep(2) # sleep 
article_p = news_soup.find("div", class_ ="article_teaser_body").text

# print both the title and the paragraph text 
print(article_title)
print(article_p)

Mars Now
NASA’s newest rover recorded audio of itself crunching over the surface of the Red Planet, adding a whole new dimension to Mars exploration.


## JPL Mars Space Images - Featured Image

- Visit the url for JPL Featured Space Image [here](https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html).
- Use splinter to navigate the site and find the image url for the current Featured Mars Image and assign the url string to a variable called featured_image_url.
- Make sure to find the image url to the full size .jpg image.

In [4]:
# go to JPL Feature Space Image
image_url = "https://data-class-jpl-space.s3.amazonaws.com/JPL_Space/index.html"
browser.visit(image_url)

# go to 'FULL IMAGE'
browser.click_link_by_partial_text('FULL IMAGE')
time.sleep(2) # sleep

# parse with BeautifulSoup
html = browser.html
image_soup = bs(html, 'html.parser')

# and scrape the URL
feature_url = image_soup.find('img', class_='fancybox-image')['src']
featured_image_url = f'https://www.jpl.nasa.gov/{feature_url}'
print(featured_image_url)

C:\Users\ocean\anaconda3\lib\site-packages\splinter\driver\webdriver\__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


https://www.jpl.nasa.gov/image/featured/mars1.jpg


## Mars Facts
- Visit the Mars Facts webpage [here](https://space-facts.com/mars/) and use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
- Use Pandas to convert the data to an HTML table string.

In [5]:
# set up url and browser
facts_url = "https://space-facts.com/mars/"
browser.visit(facts_url)

# use Pandas to parse facts url to find all tables
facts_tables = pd.read_html(facts_url)

# select the correct table from the list of tables
facts_df = facts_tables[0]

# rename the columns with appropriate headings
facts_df.columns = ['Variable', 'Value']

# convert the data to an HTML string
facts_string = facts_df.to_html(index=False)
facts_string

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>Variable</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <td>Surface Temperature:</td>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <td>First Record:</td>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <td>Recorded By:</td>\n      <td>Egyptian astronomers</td>\n    </tr>\n  </tbody>\n</table>'

## Mars Hemispheres
- Visit the USGS Astrogeology site [here](https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.
- Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. Use a Python dictionary to store the data using the keys img_url and title.
- Append the dictionary with the image url string and the hemisphere title to a list. This list contains one dictionary for each hemisphere.